<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/john210808/covasim_delta/blob/main/demo/test/CalibrateTest.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
pip install covasim optuna

In [ ]:
import pandas as pd
from datetime import datetime
import covasim as cv

pd.read_csv("https://raw.githubusercontent.com/M3IT/COVID-19_Data/master/Data/COVID_AU_state.csv").to_csv("COVID_AU_state.csv")

def processDate(start_date):
    df = pd.read_csv("COVID_AU_state.csv")
    df['date'] = pd.to_datetime(df['date'])  
    mask = (df.date >= pd.to_datetime(start_date)) & (df.state_abbrev == 'NSW')
    df = df.loc[mask]
    initCase = df['confirmed_cum'].values[0]
    df = df[['date', 'confirmed', 'tests', 'deaths']]
    df.columns = ['date', 'new_diagnoses', 'new_tests', 'new_deaths']
    df.to_csv("nsw.csv", index=False)
    return initCase
processDate('2021-06-01')

### refer to:  https://github.com/Jasminapg/Covid-19-Analysis

In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
import sciris as sc
import covasim as cv
import optuna as op
#pl.switch_backend('agg')

# refer to: http://www.healthstats.nsw.gov.au/indicator/dem_pop_age
nsw = {
   '0-9':  1057247,
  '10-19': 965844,
  '20-29': 1057444,
  '30-39': 1179944,
  '40-49': 1061300,
  '50-59': 999078,
  '60-69': 877865,
  '70-79': 617866,
  '80+':   369288
}

cv.data.country_age_data.data['NSW'] = nsw


def create_sim(x):

    beta = x[0]
    pop_infected = x[1]
    s_prob_may = x[2]
    s_prob_june = x[3]

    start_day = '2021-06-01'
    end_day   = '2021-09-15'
    data_path = 'nsw.csv'


    pars = dict(    
        location = 'NSW', 
        pop_type  = 'hybrid',
        pop_size  = 81.85e3, # 8.185M people
        pop_scale = 100,
        pop_infected = 50, 
        rescale   = True,
        rescale_threshold = 0.05,
        rescale_factor = 2,
        start_day = start_day,
        end_day   = end_day,
        contacts  = {'h':3.0, 's':20, 'w':20, 'c':20},
        beta      = 0.023,
    )
    
    # Create the baseline simulation
    sim = cv.Sim(pars=pars, datafile=data_path)
    sim['prognoses']['sus_ORs'][0] = 1 # ages 0-10
    sim['prognoses']['sus_ORs'][1] = 1 # ages 10-20

    beta_days = ['2021-06-14', '2021-06-16', '2021-06-23', '2021-06-30', '2021-07-15', '2021-08-01', '2021-08-15', '2020-08-24']

    #June opening with society opening and masks in community from 15th June reducing transmission by 15%
    
    h_beta_changes = [1.00, 1.00, 1.29, 1.29, 1.29, 1.00, 1.00, 1.29]
    s_beta_changes = [1.00, 0.90, 0.02, 0.02, 0.02, 0.23, 0.38, 0.00]
    w_beta_changes = [0.90, 0.80, 0.20, 0.20, 0.20, 0.40, 0.50, 0.465]
    c_beta_changes = [0.90, 0.80, 0.20, 0.20, 0.20, 0.40, 0.50, 0.425]

    # Define the beta changes
    h_beta = cv.change_beta(days=beta_days, changes=h_beta_changes, layers='h')
    s_beta = cv.change_beta(days=beta_days, changes=s_beta_changes, layers='s')
    w_beta = cv.change_beta(days=beta_days, changes=w_beta_changes, layers='w')
    c_beta = cv.change_beta(days=beta_days, changes=c_beta_changes, layers='c')

    #next line to save the intervention
    interventions = [h_beta, w_beta, s_beta, c_beta]

#     tc_day = sim.day('2021-03-16') #intervention of some testing (tc) starts on 16th March and we run until 1st April when it increases
#     te_day = sim.day('2021-04-01') #intervention of some testing (te) starts on 1st April and we run until 1st May when it increases
#     tt_day = sim.day('2021-05-01') #intervention of increased testing (tt) starts on 1st May
    tti_day= sim.day('2021-06-01') #intervention of tracing and enhanced testing (tti) starts on 1st June
    tti_day_july= sim.day('2021-07-01')

    # Tracing and enhanced testing strategy of symptimatics from 1st June
    #testing in June remains the same as before June under this scenario
    s_prob_march = 0.009
    s_prob_april = 0.012
    s_prob_may   = 0.012
    #no change in daily symptmatic probability in this scenario
    s_prob_june = 0.012
    t_delay       = 1.0

    iso_vals = [{k:0.1 for k in 'hswc'}]

    t_eff_june   = 0.42
    t_eff_july   = 0.47
    t_probs_june = {k:t_eff_june for k in 'hwsc'}
    t_probs_july = {k:t_eff_july for k in 'hwsc'}
    trace_d_1      = {'h':0, 's':1, 'w':1, 'c':2}

    #testing and isolation intervention
    interventions += [
#         cv.test_prob(symp_prob=0.009, asymp_prob=0.0, symp_quar_prob=0.0, asymp_quar_prob=0.0, start_day=tc_day, end_day=te_day-1, test_delay=t_delay),
#         cv.test_prob(symp_prob=s_prob_april, asymp_prob=0.0, symp_quar_prob=0.0, asymp_quar_prob=0.0, start_day=te_day, end_day=tt_day-1, test_delay=t_delay),
#         cv.test_prob(symp_prob=s_prob_may, asymp_prob=0.00075, symp_quar_prob=0.0, asymp_quar_prob=0.0, start_day=tt_day, end_day=tti_day-1, test_delay=t_delay),
        cv.test_prob(symp_prob=s_prob_june, asymp_prob=0.00075, symp_quar_prob=0.0, asymp_quar_prob=0.0, start_day=tti_day, end_day=tti_day_july-1, test_delay=t_delay),
#         cv.test_prob(symp_prob=s_prob_july, asymp_prob=0.00075, symp_quar_prob=0.0, asymp_quar_prob=0.0, start_day=tti_day_july, test_delay=t_delay),
#         cv.dynamic_pars({'iso_factor': {'days': te_day, 'vals': iso_vals}}),
        cv.contact_tracing(trace_probs=t_probs_june, trace_time=trace_d_1, start_day=tti_day, end_day=tti_day_july-1),
        cv.contact_tracing(trace_probs=t_probs_july, trace_time=trace_d_1, start_day=tti_day_july),
      ]

    sim.update_pars(interventions=interventions)
    for intervention in sim['interventions']:
        intervention.do_plot = False

    return sim



def objective(x):
    ''' Define the objective function we are trying to minimize '''

    # Create and run the sim
    sim = create_sim(x)
    sim.run()
    fit = sim.compute_fit()

    return fit.mismatch


def get_bounds():
    ''' Set parameter starting points and bounds '''
    pdict = sc.objdict(
        beta         = dict(best=0.00593, lb=0.0059, ub=0.006),
        pop_infected = dict(best=1500,  lb=1000,   ub=1600),
        s_prob_may = dict(best=0.0171,  lb=0.016,   ub=0.019),
        s_prob_june = dict(best=0.0171,  lb=0.016,   ub=0.019),
    )

    # Convert from dicts to arrays
    pars = sc.objdict()
    for key in ['best', 'lb', 'ub']:
        pars[key] = np.array([v[key] for v in pdict.values()])

    return pars, pdict.keys()


#%% Calibration

name      = 'covasim_calibration'
storage   = f'sqlite:///{name}.db'
n_trials  = 10
n_workers = 4

pars, pkeys = get_bounds() # Get parameter guesses


def op_objective(trial):

    pars, pkeys = get_bounds() # Get parameter guesses
    x = np.zeros(len(pkeys))
    for k,key in enumerate(pkeys):
        x[k] = trial.suggest_uniform(key, pars.lb[k], pars.ub[k])

    return objective(x)


def worker():
    study = op.load_study(storage=storage, study_name=name)
    return study.optimize(op_objective, n_trials=n_trials)


def run_workers():
    return sc.parallelize(worker, n_workers)


def make_study():
    try: op.delete_study(storage=storage, study_name=name)
    except: pass
    return op.create_study(storage=storage, study_name=name)


def calibrate():
    ''' Perform the calibration '''
    make_study()
    run_workers()
    study = op.load_study(storage=storage, study_name=name)
    output = study.best_params
    return output, study


def savejson(study):
    dbname = 'calibrated_parameters'

    sc.heading('Making results structure...')
    results = []
    failed_trials = []
    for trial in study.trials:
        data = {'index':trial.number, 'mismatch': trial.value}
        for key,val in trial.params.items():
            data[key] = val
        if data['mismatch'] is None:
            failed_trials.append(data['index'])
        else:
            results.append(data)
    print(f'Processed {len(study.trials)} trials; {len(failed_trials)} failed')

    sc.heading('Making data structure...')
    keys = ['index', 'mismatch'] + pkeys
    data = sc.objdict().make(keys=keys, vals=[])
    for i,r in enumerate(results):
        for key in keys:
            data[key].append(r[key])
    df = pd.DataFrame.from_dict(data)

    order = np.argsort(df['mismatch'])
    json = []
    for o in order:
        row = df.iloc[o,:].to_dict()
        rowdict = dict(index=row.pop('index'), mismatch=row.pop('mismatch'), pars={})
        for key,val in row.items():
            rowdict['pars'][key] = val
        json.append(rowdict)
    sc.savejson(f'{dbname}.json', json, indent=2)

    return


if __name__ == '__main__':

    do_save = True

    to_plot = ['cum_infections', 'new_infections', 'cum_tests', 'new_tests', 'cum_diagnoses', 'new_diagnoses', 'cum_deaths', 'new_deaths']

    # # Plot initial
    print('Running initial...')
    pars, pkeys = get_bounds() # Get parameter guesses
    sim = create_sim(pars.best)
    sim.run()
    sim.plot(to_plot=to_plot)
    #pl.gcf().axes[0].set_title('Initial parameter values')
    objective(pars.best)
    pl.pause(1.0) # Ensure it has time to render

    # Calibrate
    print('Starting calibration for {state}...')
    T = sc.tic()
    pars_calib, study = calibrate()
    sc.toc(T)

    # Plot result
    print('Plotting result...')
    sim = create_sim([pars_calib['beta'], pars_calib['pop_infected'], pars_calib['s_prob_may'], pars_calib['s_prob_june']])
    sim.run()
    sim.plot(to_plot=to_plot)
    pl.gcf().axes[0].set_title('Calibrated parameter values')

    if do_save:
        savejson(study)


print('Done.')